In [191]:
import time
import pickle

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup

import plotly.graph_objects as go

import pandas as pd

from datetime import datetime

In [100]:
# All_fight_data = [Events] Ordered from old -> new.
    # Event = [Fights, Date, Venue]
        # Fight = [Fighter1, Fighter2, Result]
            # Fighter1, Fighter2 = [Name, Total, Strikes]
                # Totals = [Total, First, Second, Third, Fourth, Fifth]
                    # Total, First, etc = [Knockdowns, Takedowns, Reversals, Submission Attempted, Control Time]
                        # Takedowns = [Landed, Attempted]
                # Strikes = [Total, First, Second, Third, Fourth, Fifth]
                    # Total, First, etc = [Significant Strikes, Head, Body, Leg, Distance, Clinch, Ground]
                        # Everything = [Landed, Attempted]
            # Result = [Winner, Method, Round, Time, Referee, Weight, Bonus, Details]

In [101]:
def open_data(name):
    with open(name, "rb") as fp:
        return pickle.load(fp)

In [102]:
def save_data(data, name):
    with open(name, "wb") as fp:
        pickle.dump(data, fp)

In [150]:
all_fight_data = open_data('all_fight_data')

In [151]:
def all_fighter_names(data):
    all_names = []
    for event in data:
        for fight in event[0]:
            if fight[0][0] not in all_names:
                all_names.append(fight[0][0])
                
            if fight[1][0] not in all_names:
                all_names.append(fight[1][0])
    return all_names

In [157]:
def all_referee_names(data):
    all_names = []
    for event in data:
        for fight in event[0]:
            if fight[2][4] not in all_names:
                all_names.append(fight[2][4])
    
    return all_names

In [195]:
def convert_date(date):
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    month = date.split(' ')[0]
    month = months.index(month) + 1
    
    day = int(date.split(',')[0].split(' ')[1])
    year = int(date.split(', ')[1])
    date = datetime(year, month, day)
    return date

In [221]:
def bar_data(data, group):
    # [name, ko, submission, decision, finish]
    if group == 'Fighters':
        return []
    else:
        all_r_names = all_referee_names(data)
        all_data = [[all_r_names[i], 0, 0, 0, 0] for i in range(len(all_r_names))]
        
        for event in data:
            for fight in event[0]:
                name_index = all_r_names.index(fight[2][4])
                    
                if fight[2][1] == 'KO/TKO':
                    all_data[name_index][1] += 1
                    all_data[name_index][4] += 1
                elif fight[2][1] == 'Submission':
                    all_data[name_index][2] += 1
                    all_data[name_index][4] += 1
                else:
                    all_data[name_index][3] += 1
                    all_data[name_index][4] -= 1
                    

    return all_data

b_data = bar_data(all_fight_data, 'Referees')
df = pd.DataFrame(b_data)

In [228]:
df[[0,1]].sort_values(1)[::-1]

,0,1
16,Herb Dean,361
0,John McCarthy,240
23,Dan Miragliotta,162
5,Mario Yamasaki,127
35,Marc Goddard,126
...,...,...
134,David Ferguson,0
34,Jim Axtel,0
136,Joe Solis,0
31,Nick Gamst,0
